In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import sqlalchemy as sa

import database as db
from images import copy_images, get_list_of_images_invalid, save_metadata
from models import County, DataTrustedIdentifier, Image
from unaccent import unaccent

In [3]:
engine, session=db.connect()
engine.echo=False

2023-03-30 10:29:54,585 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-30 10:29:54,586 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-30 10:29:54,587 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-30 10:29:54,587 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-30 10:29:54,588 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-30 10:29:54,588 INFO sqlalchemy.engine.Engine [raw sql] {}


In [4]:
list_minimum_image=[5, 10, 20]
list_color=['rgb', 'grayscale']
list_image_size=['256', '400', '512']
list_regions=['Norte', 'Nordeste', 'Centro-Oeste', 'Sul', 'Sudeste']
path_out = '/home/xandao/Imagens/region_dataset'
list_images_invalid=get_list_of_images_invalid()
list_images_invalid=list_images_invalid['barcode']
list_images_invalid

['INPA0248526',
 'INPA0248523',
 'INPA0248528',
 'NY01421926_01',
 'NY01421575_01',
 'HUFSJ001689_v00',
 'HUFSJ001133_v00',
 'HUFSJ002198_v00',
 'HUFSJ003255_v00',
 'HVASF000487_v01',
 'HUFSJ001689_v01',
 'INPA0019084_nd',
 'INPA0022379_nd',
 'INPA0032742_nd',
 'INPA0023115',
 'NL-U1484137',
 'NY01397568_01',
 'INPA0012286',
 'INPA0146998']

In [5]:
for color in list_color:
    for region in list_regions:
        for image_size in list_image_size:
            for minimum_image in list_minimum_image:
                query = session.query(County).filter(County.regiao.__eq__(region)).distinct().all()

                list_uf = [unaccent(sa.func.lower(q.uf)) for q in query]
                list_state = [unaccent(sa.func.lower(q.state)) for q in query]
                list_county = [unaccent(sa.func.lower(q.county)) for q in query]

                uf_unaccented_lower = unaccent(sa.func.lower(DataTrustedIdentifier.state_province)).in_(list_uf)
                state_unaccented_lower = unaccent(sa.func.lower(DataTrustedIdentifier.state_province)).in_(list_state)
                county_unaccented_lower = unaccent(sa.func.lower(DataTrustedIdentifier.county)).in_(list_county)

                list_level = []
                list_path_images_final = []
                list_seq_final = []
                list_count_samples = []

                level=DataTrustedIdentifier.specific_epithet_trusted
                out=os.path.join(path_out, color.upper(), level.name, region, image_size, str(minimum_image))

                condition = sa.and_(DataTrustedIdentifier.country_trusted == 'Brasil',
                                    level.is_not(None),
                                    sa.or_(uf_unaccented_lower,
                                           state_unaccented_lower,
                                           county_unaccented_lower))

                columns = [level,
                           sa.func.array_agg(DataTrustedIdentifier.seq).label('list_seq')]
                query = session.query(*columns) \
                    .filter(condition) \
                    .distinct() \
                    .group_by(level) \
                    .order_by(level) \
                    .having(sa.func.count(level) >= minimum_image) \
                    .all()

                print('region: %s color: %s image_size: %s minimum_image: %d' % (region, color.upper(), image_size, minimum_image))
                for q in query:
                    qzao = session.query(Image.seq_id, sa.func.array_agg(Image.path_segmented).label('list_path_segmented')) \
                        .filter(sa.and_(Image.seq_id.in_(q.list_seq),
                                        Image.color_mode.__eq__(color.upper()),
                                        Image.height.__eq__(image_size),
                                        Image.width.__eq__(image_size),
                                        sa.not_(Image.filename.in_(list_images_invalid)),
                                        sa.not_(Image.filename.ilike('%_v0%')),
                                        sa.not_(Image.filename.like('%_e%')),
                                        sa.not_(Image.filename.like('%_nd%')),
                                        ))\
                        .group_by(Image.seq_id)\
                        .all()

                    list_one_image_per_seq = []
                    list_seq_one_image = []
                    for qzinho in qzao:
                        list_seq_one_image.append(qzinho.seq_id)
                        list_one_image_per_seq.append(sorted(qzinho.list_path_segmented)[0])

                    if len(list_one_image_per_seq) >= minimum_image:
                        list_count_samples.append(len(list_one_image_per_seq))
                        list_level.append(q.specific_epithet_trusted)
                        list_path_images_final.append(list_one_image_per_seq)
                        list_seq_final.append(list_seq_one_image)

                copy_images(list_level, list_path_images_final, out)
                save_metadata(list_count_samples, list_level, list_path_images_final, list_seq_final, out, session)

/tmp/ipykernel_40315/2129993866.py:31: SAWarning: Class unaccent will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/20/cprf)
  query = session.query(*columns) \


region: Norte color: RGB image_size: 256 minimum_image: 5
total of levels: 110 total of images: 2682


,levels,paths,count,seq
0,aduncum,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,58,"[8714, 9521, 11342, 11874, 11876, 11893, 12507..."
1,aequale,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,9,"[12577, 30534, 30546, 36793, 43763, 43769, 457..."
2,alata,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,8,"[26378, 26385, 26457, 37031, 43122, 43125, 431..."
3,alatabaccum,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,28,"[26207, 30555, 30557, 30558, 30560, 30561, 305..."


,seq,genus,specific_epithet,genus_trusted,specific_epithet_trusted,urls
0,8714,Piper,aduncum,Piper,aduncum,[https://storage.googleapis.com/cria-zoomify/h...
1,1363,Peperomia,glabella,Peperomia,glabella,[https://storage.googleapis.com/cria-zoomify/h...
2,44254,Piper,coruscans,Piper,coruscans,[https://storage.googleapis.com/cria-zoomify/h...
3,13605,Piper,arboreum,Piper,arboreum,[https://storage.googleapis.com/cria-zoomify/h...


region: Norte color: RGB image_size: 256 minimum_image: 10
total of levels: 69 total of images: 2387


...

,levels,paths,count,seq
0,aduncum,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,94,"[7742, 7757, 8700, 8705, 8711, 9435, 9439, 957..."
1,alata,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,29,"[6857, 6861, 6862, 6863, 6864, 7565, 7859, 110..."
2,amalago,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,114,"[2798, 7741, 7908, 7928, 8751, 8757, 8758, 876..."
3,amplum,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,60,"[7834, 7864, 8785, 8792, 9387, 9401, 9443, 957..."


,seq,genus,specific_epithet,genus_trusted,specific_epithet_trusted,urls
0,19770,Ottonia,anisum,Piper,anisum,[https://storage.googleapis.com/cria-zoomify/h...
1,7333,Piper,corcovadense,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...
2,21142,Ottonia,propinqua,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...
3,21260,Piper,corcovadensis,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...


region: Sudeste color: GRAYSCALE image_size: 512 minimum_image: 20
total of levels: 42 total of images: 2263


,levels,paths,count,seq
0,aduncum,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,94,"[7742, 7757, 8700, 8705, 8711, 9435, 9439, 957..."
1,alata,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,29,"[6857, 6861, 6862, 6863, 6864, 7565, 7859, 110..."
2,amalago,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,114,"[2798, 7741, 7908, 7928, 8751, 8757, 8758, 876..."
3,amplum,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,60,"[7834, 7864, 8785, 8792, 9387, 9401, 9443, 957..."


,seq,genus,specific_epithet,genus_trusted,specific_epithet_trusted,urls
0,19770,Ottonia,anisum,Piper,anisum,[https://storage.googleapis.com/cria-zoomify/h...
1,7333,Piper,corcovadense,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...
2,21142,Ottonia,propinqua,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...
3,21260,Piper,corcovadensis,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...


In [6]:
session.close()
engine.dispose()